<a href="https://colab.research.google.com/github/virendrasinh734/skin_disease_detection_using_cnns/blob/main/collections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install wget
!wget https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.
--2023-09-23 10:44:33--  https://isic-challenge-data.s3.amazonaws.com/2019/ISIC_2019_Training_Input.zip
Resolving isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)... 3.5.8.13, 52.217.37.100, 52.217.232.201, ...
Connecting to isic-challenge-data.s3.amazonaws.com (isic-challenge-data.s3.amazonaws.com)|3.5.8.13|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9771618190 (9.1G) [application/zip]
Saving to: ‘ISIC_2019_Training_Input.zip’

ISIC_2019_Training_ 100%[===================>]   9.10G  68.2MB/s    in 2m 16s  

2023-09-23 10:46:50 (68.4 MB/s) - ‘ISIC_2019_Training_Input.zip’ saved [9771618190/9771618190]



In [2]:
!unzip ISIC_2019_Training_Input.zip

Streaming output truncated to the last 5000 lines.
  inflating: ISIC_2019_Training_Input/ISIC_0065357.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065358.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065359.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065360.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065361.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065362.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065366.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065367.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065368.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065370.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065371.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065372.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065374.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065376.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065377.jpg  
  inflating: ISIC_2019_Training_Input/ISIC_0065378.jpg  
  inflating: ISIC_2019_Training_Input

In [8]:
import pandas as pd
import numpy as np
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.applications import VGG16
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Conv2D, MaxPool2D

# Load the labels CSV
labels_df = pd.read_csv('/content/result.csv')
labels_df = labels_df[:15000]
class_labels = labels_df.columns[1:]
image_filenames = labels_df['image']
for i in range(len(image_filenames)):
    image_filenames[i] = image_filenames[i] + ".jpg"

labels = labels_df[class_labels].values
X_train, X_val, y_train, y_val = train_test_split(image_filenames, labels, test_size=0.2, random_state=42)

# Define a custom data generator for preprocessing
def custom_data_generator(image_paths, labels, batch_size, target_size=(224, 224)):
    num_samples = len(image_paths)
    while True:
        for i in range(0, num_samples, batch_size):
            batch_image_paths = image_paths[i:i + batch_size]
            batch_labels = labels[i:i + batch_size]
            batch_images = [preprocess_image(image_path, target_size) for image_path in batch_image_paths]
            yield np.array(batch_images), batch_labels

def preprocess_image(image_path, target_size=(224, 224)):
    img = Image.open('/content/ISIC_2019_Training_Input/' + image_path)
    img = img.resize(target_size)
    img_array = np.array(img) / 255.0
    return img_array

batch_size = 32
train_data_generator = custom_data_generator(X_train, y_train, batch_size)
val_data_generator = custom_data_generator(X_val, y_val, batch_size)




<ipython-input-8-21cfbe8025ed>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  image_filenames[i] = image_filenames[i] + ".jpg"


In [5]:
# Load the pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(256, activation='relu'))
model.add(Dense(9, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

epochs = 12
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

history = model.fit(train_data_generator,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=val_data_generator,
                    validation_steps=validation_steps)

58889256/58889256 [==============================] - 0s 0us/step
Epoch 1/12
100/100 [==============================] - 170s 2s/step - loss: 1.2821 - accuracy: 0.6486 - val_loss: 0.9457 - val_accuracy: 0.6712
Epoch 2/12
100/100 [==============================] - 143s 1s/step - loss: 0.9444 - accuracy: 0.6925 - val_loss: 0.9128 - val_accuracy: 0.6775
Epoch 3/12
100/100 [==============================] - 134s 1s/step - loss: 0.9115 - accuracy: 0.6941 - val_loss: 0.9075 - val_accuracy: 0.6819
Epoch 4/12
100/100 [==============================] - 138s 1s/step - loss: 0.8930 - accuracy: 0.7000 - val_loss: 0.8828 - val_accuracy: 0.6875
Epoch 5/12
100/100 [==============================] - 172s 2s/step - loss: 0.8693 - accuracy: 0.7019 - val_loss: 0.8667 - val_accuracy: 0.6781
Epoch 6/12
100/100 [==============================] - 154s 2s/step - loss: 0.8493 - accuracy: 0.7131 - val_loss: 0.8894 - val_accuracy: 0.6894
Epoch 7/12
100/100 [==============================] - 123s 1s/step - loss: 0.

In [19]:
from PIL import Image
import numpy as np

# Load and preprocess an image for inference
input_image = Image.open('/content/ISIC_2019_Training_Input/ISIC_0000001.jpg')
input_image = input_image.resize((224, 224))  # Resize to (224, 224)
input_image = np.array(input_image)  # Convert to NumPy array
input_image = input_image / 255.0  # Normalize pixel values
input_image = np.expand_dims(input_image, axis=0)  # Add batch dimension
predictions = model.predict(input_image)
s=np.argmax(predictions)
print(predictions)
print(s)

1/1 [==============================] - 1s 502ms/step
[[8.6816400e-02 8.7859291e-01 1.2929298e-03 2.4020375e-05 3.0403038e-02
  7.4301922e-04 1.7376830e-03 3.9000076e-04 4.0077635e-09]]
1


In [6]:
model.save('my_model.h5')
from google.colab import files
files.download('my_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
from keras.applications import ResNet50
from keras.layers import GlobalAveragePooling2D
# Load the pre-trained ResNet50 model
base_model2 = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Specify the number of layers from the end to unfreeze
num_layers_to_unfreeze = 6  # You can adjust this number as needed

# Unfreeze the specified layers
for layer in base_model2.layers[:-num_layers_to_unfreeze]:
    layer.trainable = False
model2 = Sequential()
model2.add(base_model)
model2.add(GlobalAveragePooling2D())
model2.add(Dense(512, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(256, activation='relu'))
model2.add(Dense(128, activation='relu'))
model2.add(Dense(9, activation='softmax'))

# Compile the model
model2.compile(loss='categorical_crossentropy',
              optimizer='Adam',  # You can adjust the learning rate as needed
              metrics=['accuracy'])

epochs = 10
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

history2 = model2.fit(train_data_generator,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=val_data_generator,
                    validation_steps=validation_steps)


Epoch 1/10
375/375 [==============================] - 306s 808ms/step - loss: 1.1323 - accuracy: 0.6324 - val_loss: 1.0227 - val_accuracy: 0.6358
Epoch 2/10
375/375 [==============================] - 279s 744ms/step - loss: 1.0452 - accuracy: 0.6442 - val_loss: 0.9811 - val_accuracy: 0.6492
Epoch 3/10
375/375 [==============================] - 266s 711ms/step - loss: 1.0099 - accuracy: 0.6518 - val_loss: 0.9661 - val_accuracy: 0.6542
Epoch 4/10
375/375 [==============================] - 264s 704ms/step - loss: 0.9911 - accuracy: 0.6549 - val_loss: 0.9547 - val_accuracy: 0.6576
Epoch 5/10
375/375 [==============================] - 266s 712ms/step - loss: 0.9729 - accuracy: 0.6594 - val_loss: 0.9549 - val_accuracy: 0.6509
Epoch 6/10
375/375 [==============================] - 278s 743ms/step - loss: 0.9622 - accuracy: 0.6580 - val_loss: 0.9351 - val_accuracy: 0.6583
Epoch 7/10
375/375 [==============================] - 255s 683ms/step - loss: 0.9531 - accuracy: 0.6637 - val_loss: 0.9294 -

In [10]:
model2.save('my_model2.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [15]:
from keras.layers import MaxPooling2D
model3= Sequential()

# Initial Convolutional Block
model3.add(Conv2D(64, (3, 3), activation="relu", input_shape=(224, 224, 3)))
model3.add(Conv2D(64, (3, 3), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(128, (3, 3), activation="relu"))
model3.add(Conv2D(128, (3, 3), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(256, (3, 3), activation="relu"))
model3.add(Conv2D(256, (3, 3), activation="relu"))
model3.add(Conv2D(256, (3, 3), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 2)))

model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(Conv2D(512, (3, 3), activation="relu"))
model3.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten Layer
model3.add(Flatten())

# Fully Connected Layers
model3.add(Dense(512, activation="relu"))
model3.add(Dropout(0.5))
model3.add(Dense(256, activation="relu"))
model3.add(Dropout(0.5))
model3.add(Dense(9, activation="softmax"))

model3.summary
# Compile the model
model3.compile(loss="categorical_crossentropy", optimizer="Adam", metrics=["accuracy"])

epochs = 10
steps_per_epoch = len(X_train) // batch_size
validation_steps = len(X_val) // batch_size

history3 = model3.fit(train_data_generator,
                    epochs=epochs,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=val_data_generator,
                    validation_steps=validation_steps)


Epoch 1/10
375/375 [==============================] - 353s 885ms/step - loss: 1.3477 - accuracy: 0.6196 - val_loss: 1.1410 - val_accuracy: 0.6176
Epoch 2/10
375/375 [==============================] - 347s 927ms/step - loss: 1.1181 - accuracy: 0.6269 - val_loss: 1.0869 - val_accuracy: 0.6176
Epoch 3/10
375/375 [==============================] - 331s 883ms/step - loss: 1.0814 - accuracy: 0.6258 - val_loss: 1.1531 - val_accuracy: 0.6176
Epoch 4/10
375/375 [==============================] - 327s 873ms/step - loss: 1.0720 - accuracy: 0.6248 - val_loss: 1.0791 - val_accuracy: 0.6085
Epoch 5/10
375/375 [==============================] - 322s 859ms/step - loss: 1.0454 - accuracy: 0.6280 - val_loss: 1.0709 - val_accuracy: 0.6153
Epoch 6/10
375/375 [==============================] - 299s 797ms/step - loss: 1.0337 - accuracy: 0.6298 - val_loss: 1.0521 - val_accuracy: 0.6169
Epoch 7/10
375/375 [==============================] - 290s 774ms/step - loss: 1.0493 - accuracy: 0.6317 - val_loss: 1.1334 -

KeyboardInterrupt: ignored